In [1]:
####### Example for loading distance matrix using basic PHYLIP format (http://www.phylo.org/tools/obsolete/phylip.html)


# utility file for reading distance matrices stored in basic PHYLIP format (http://www.phylo.org/tools/obsolete/phylip.html)

def parse(distance_matrix):
    """Read matrix of distances form Phylip-formatted file.(http://www.phylo.org/tools/obsolete/phylip.html)

    Return list of names and dictionary structure such that
    d[a][b] is the distance between names[a] and names[b].
    """
    try:
        #with open(filename) as src:
        #    lines = [line.strip() for line in src if line.strip()]
        lines = distance_matrix.split("\n")
        try:
            n = int(lines[0])
        except ValueError:
            raise ValueError('First line of file should contain number of samples')

        if n < 2:
            raise ValueError('N must be 2 or more')

        if len(lines) < 1+n:
            raise ValueError('Not enough lines of data')

        names = [line.split()[0] for line in lines[1:1+n]]
        dist = []
        for k in range(n):
            pieces = lines[1+k].split()[1:]
            try:
                vals = [float(v) for v in pieces]
            except ValueError:
                raise ValueError('invalid distance on line ' + str(2+k))

            if len(vals) != n:
                raise ValueError('line %d should have %d entries' % (2+k,n))

            dist.append( { j : vals[j] for j in range(n) } )
    except IOError:
        print('Unable to open file:',filename)
        raise

    return names,dist

In [22]:
####### Block 4: Do not change anything in this block  #######
def clustering_main(names, dist, compute_distance):
    #--------------------------------------------------------
    # Initialize Data Structures to save clusters, trees, and nodes
    #--------------------------------------------------------

    # clusters[k] is a list of original leaf IDs for those
    # leaves within cluster k, i.e. [[1], [2], [3]]
    clusters = []
    for k in range(N):
        clusters.append( [k] )   # cluster k originally has only leaf k


    # tree[k] is our tuple representation of cluster k, i.e., ('A',(),())
    trees = []
    for k in range(N):
        trees.append( (names[k], (), ()) )



    # originally, clusters 0 through N-1 are the active ones to merge and calculate distance
    activeClusters = set( range(N) )


    #--------------------------------------------------------
    # Main clustering algorithm
    #--------------------------------------------------------

    ##### Keep merging clusters until one cluster left
    while len(activeClusters) > 1:

        #### step 1: find the 'closest' pair from distance matrix and return index. 
        #### Need define function   find_nearest_active()

        a,b = find_nearest_active(dist, activeClusters)


        #### step 2: remove a and b from active set, a and b is the index of node
        activeClusters.remove(a)
        activeClusters.remove(b)

        #### reporting merging information
        print('About to merge clusters at distance %.3f:' % dist[a][b])
        print('  First:  ' + str(trees[a]))
        print('  Second: ' + str(trees[b]))


        # new cluster has all leaves of clusters[a] and clusters[b]
        clusters.append(clusters[a] + clusters[b])  # combination of both subtree, i.e.  [1] + [2] = [1,2]

        #### step 4: create new tree representation for this cluster. 
        #### Need define function     combine_trees()
        trees.append( combine_trees(dist, trees,a,b) )

        print('  Result: ', trees[-1])

        #### step 5: compute distances between all other clusters to the new clade
        newID = len(clusters)-1

        dist.append( {} )    # dictionary for new cluster's distances to other clusters
        for c in activeClusters:
            val = compute_distance(dist, clusters,c, newID) #### Need define compute_distance function
            dist[newID][c] = val
            dist[c][newID] = val

        print('  Distances to new cluster:')
        for c in activeClusters:
            print('    %.2f: %s' % (dist[newID][c], trees[c]))

        #### step 6: officially add new cluster to active set, and repeat the procedure
        activeClusters.add(newID)
    
    final_tree = trees[-1]
    
    print("Final tree using single linkage:",) 
    return final_tree
    

In [27]:
#--------------------------------------------------------
# Utility functions (to be completed by student)
#--------------------------------------------------------


#--------------------------------------------------------
# Summary of data structures (created later in this script).
# All indexing is ZERO-based (i.e. names[0] is the first leaf)
#
# names[k] is the original name of the k-th leaf of the input
#
# clusters[k] is a list of integers that ID the leaves of the cluster
#
# trees[k] is a tuple representation of the subtree for clades[k]
#
# activeClusters is a list of integer IDs for active clusters during the
# clustering process
#
# dist is list of dictionaries such that dist[a][b] is distance
# between clusters a and b
#
#--------------------------------------------------------


def combine_trees(dist,trees, a,b):
  combine = (dist[a][b]/2,trees[a],trees[b])
  return combine
  #Remember to divde combine by 2
    
    ##Return tuple that represents tree for newly merged clusters a and b.
    ##For instance, a is ('A',(),()), b is ('B',()),()), and the distance between A and B in dist matrix is 0.5,
    ##The combined tree should be (0.5,('A',(),()), ('B',(),()))
    ##You need write the correct codes for this function.
    

def find_nearest_active(dist,activeClusters):
  a = float('inf')
  for x in activeClusters:
    for y in activeClusters:
      if x > y:
        if dist[x][y]!= 0 and dist[x][y] < a:
         a = dist[x][y]
         i = x
         j = y
        # x is row and y is column
  return i,j
    ##Return IDs (index) for the two nearest active clusters in the matrix. 
    ##For instance, if i-th row and j-th column give the smallest number, just return i,j

def single_linkage_cost(dist, clusters, a,b):
  single = float('inf')
  z = dist
  for x in clusters[b]:
    for y in clusters[a]:
       if z[x][y] != 0 and z[x][y] < single:
        single = z[x][y]
  return single
  
    ##Return distance between clusters a and b.
    ##For single linkage, distance should be based on the
    ##NEAREST neighbors across the clusters.

def complete_linkage_cost(dist, clusters, a, b):
  complete = 0
  z = dist
  for x in clusters[b]:
    for y in clusters[a]:
      if z[x][y] != 0 and z[x][y] > complete:
       complete = z[x][y]
  return complete
    ##Return distance between clusters a and b.
    ##For complete linkage, distance should be based on the
    ##FURTHEST neighbors across the clusters.

def upgma_linkage_cost(dist, clusters, a, b):
  z = dist
  n=0
  m=0
  for x in clusters[b]:
    for y in clusters[a]:
     n += dist[x][y] 
     m += 1
     upgma = n/m
  return upgma

  ##Return distance between clusters a and b. For UPGMA, distance should be the unweighted average of 
  ##distances for all pairs across the clusters.    

In [28]:
####### example: Load distance matrix

dist = """5
A 0	20	60	100	90
B 20	0	50	90	80
C 60	50	0	40	50
D 100	90	40	0	30
E 90	80	50	30	0"""

names,dist = parse(dist)
N = len(names)               # number of leaves

print("Number of leaves: ", N)
print("The leaf nodes: ", names)
print("Distance matrix: ", dist)

Number of leaves:  5
The leaf nodes:  ['A', 'B', 'C', 'D', 'E']
Distance matrix:  [{0: 0.0, 1: 20.0, 2: 60.0, 3: 100.0, 4: 90.0}, {0: 20.0, 1: 0.0, 2: 50.0, 3: 90.0, 4: 80.0}, {0: 60.0, 1: 50.0, 2: 0.0, 3: 40.0, 4: 50.0}, {0: 100.0, 1: 90.0, 2: 40.0, 3: 0.0, 4: 30.0}, {0: 90.0, 1: 80.0, 2: 50.0, 3: 30.0, 4: 0.0}]


In [29]:
clustering_main(names, dist, single_linkage_cost)

About to merge clusters at distance 20.000:
  First:  ('B', (), ())
  Second: ('A', (), ())
  Result:  (10.0, ('B', (), ()), ('A', (), ()))
  Distances to new cluster:
    50.00: ('C', (), ())
    90.00: ('D', (), ())
    80.00: ('E', (), ())
About to merge clusters at distance 30.000:
  First:  ('E', (), ())
  Second: ('D', (), ())
  Result:  (15.0, ('E', (), ()), ('D', (), ()))
  Distances to new cluster:
    40.00: ('C', (), ())
    80.00: (10.0, ('B', (), ()), ('A', (), ()))
About to merge clusters at distance 40.000:
  First:  (15.0, ('E', (), ()), ('D', (), ()))
  Second: ('C', (), ())
  Result:  (20.0, (15.0, ('E', (), ()), ('D', (), ())), ('C', (), ()))
  Distances to new cluster:
    50.00: (10.0, ('B', (), ()), ('A', (), ()))
About to merge clusters at distance 50.000:
  First:  (20.0, (15.0, ('E', (), ()), ('D', (), ())), ('C', (), ()))
  Second: (10.0, ('B', (), ()), ('A', (), ()))
  Result:  (25.0, (20.0, (15.0, ('E', (), ()), ('D', (), ())), ('C', (), ())), (10.0, ('B', (

(25.0,
 (20.0, (15.0, ('E', (), ()), ('D', (), ())), ('C', (), ())),
 (10.0, ('B', (), ()), ('A', (), ())))

In [30]:
clustering_main(names, dist, complete_linkage_cost)

About to merge clusters at distance 20.000:
  First:  ('B', (), ())
  Second: ('A', (), ())
  Result:  (10.0, ('B', (), ()), ('A', (), ()))
  Distances to new cluster:
    60.00: ('C', (), ())
    100.00: ('D', (), ())
    90.00: ('E', (), ())
About to merge clusters at distance 30.000:
  First:  ('E', (), ())
  Second: ('D', (), ())
  Result:  (15.0, ('E', (), ()), ('D', (), ()))
  Distances to new cluster:
    50.00: ('C', (), ())
    100.00: (10.0, ('B', (), ()), ('A', (), ()))
About to merge clusters at distance 50.000:
  First:  (15.0, ('E', (), ()), ('D', (), ()))
  Second: ('C', (), ())
  Result:  (25.0, (15.0, ('E', (), ()), ('D', (), ())), ('C', (), ()))
  Distances to new cluster:
    100.00: (10.0, ('B', (), ()), ('A', (), ()))
About to merge clusters at distance 100.000:
  First:  (25.0, (15.0, ('E', (), ()), ('D', (), ())), ('C', (), ()))
  Second: (10.0, ('B', (), ()), ('A', (), ()))
  Result:  (50.0, (25.0, (15.0, ('E', (), ()), ('D', (), ())), ('C', (), ())), (10.0, ('B

(50.0,
 (25.0, (15.0, ('E', (), ()), ('D', (), ())), ('C', (), ())),
 (10.0, ('B', (), ()), ('A', (), ())))

In [31]:
clustering_main(names, dist, upgma_linkage_cost)

About to merge clusters at distance 20.000:
  First:  ('B', (), ())
  Second: ('A', (), ())
  Result:  (10.0, ('B', (), ()), ('A', (), ()))
  Distances to new cluster:
    55.00: ('C', (), ())
    95.00: ('D', (), ())
    85.00: ('E', (), ())
About to merge clusters at distance 30.000:
  First:  ('E', (), ())
  Second: ('D', (), ())
  Result:  (15.0, ('E', (), ()), ('D', (), ()))
  Distances to new cluster:
    45.00: ('C', (), ())
    90.00: (10.0, ('B', (), ()), ('A', (), ()))
About to merge clusters at distance 45.000:
  First:  (15.0, ('E', (), ()), ('D', (), ()))
  Second: ('C', (), ())
  Result:  (22.5, (15.0, ('E', (), ()), ('D', (), ())), ('C', (), ()))
  Distances to new cluster:
    78.33: (10.0, ('B', (), ()), ('A', (), ()))
About to merge clusters at distance 78.333:
  First:  (22.5, (15.0, ('E', (), ()), ('D', (), ())), ('C', (), ()))
  Second: (10.0, ('B', (), ()), ('A', (), ()))
  Result:  (39.166666666666664, (22.5, (15.0, ('E', (), ()), ('D', (), ())), ('C', (), ())), 

(39.166666666666664,
 (22.5, (15.0, ('E', (), ()), ('D', (), ())), ('C', (), ())),
 (10.0, ('B', (), ()), ('A', (), ())))